In [ ]:
# 1. download futures data daily, hourly, minute bar
# 2. v5 API (bybit price and mark price), 做切割

In [1]:
import requests 
import json
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import time
import pybit

In [2]:
sdate = dt.date(2020,1,15)
edate = dt.date.today()

# this will give you a list containing all of the dates
dates = [dt.datetime.strftime(sdate + dt.timedelta(days=x), '%Y-%m-%d') for x in range((edate-sdate).days + 1)]

In [3]:
# old API
def get_bar_data(symbol, interval, startTime, exchange='bybit'):
    if exchange.lower() == 'bybit':
        url = "https://api.bybit.com/public/linear/kline"
    else:
        url = "https://api.bybit.com/public/linear/mark-price-kline"
    req_params = {"symbol" : symbol, 'interval' : interval, 'from' : startTime}
    df = pd.DataFrame(json.loads(requests.get(url, params = req_params).text)['result'])
    if (len(df.index) == 0):
        return None
    if exchange=='bybit':
        df.index = [dt.datetime.fromtimestamp(x) for x in df.open_time]
    else:
        df.index = [dt.datetime.fromtimestamp(x) for x in df.start_at]
    return df

In [15]:
# convert date or datetime to seconds, as one argument for v5 API
def dt_to_millsec(date, dateformat='%Y-%m-%d'):
    if len(date)==19 and dateformat=='%Y-%m-%d':
        dateformat = '%Y-%m-%d %H:%M:%S'
    t = dt.datetime.strptime(date, dateformat)
    return int(time.mktime(t.timetuple())*1000)

In [17]:
def get_bar_data_v5(category, symbol, interval, startTime=None):
    url = "https://api.bybit.com/v5/market/kline"
    if startTime is None:
        req_params = {"category": category, "symbol" : symbol, 'interval' : interval}
    else:
        req_params = {"category": category, "symbol" : symbol, 'interval' : interval, 'start' : startTime}
    df = pd.DataFrame(json.loads(requests.get(url, params = req_params).text)['result'])
    if (len(df.index)==0):
        return None
    df = pd.concat([df['symbol'], pd.DataFrame(list(df['list']), columns=['startTime', 'openPrice', 'highPrice', 'lowPrice', 'closePrice', 'volume', 'turnover'])], axis=1)
    df.index = [dt.datetime.fromtimestamp(int(x)/1000) for x in df.startTime]
    df = df.iloc[::-1]
    return df

In [6]:
def find_sdate(category, symbol, interval):
    sindex = 0
    tindex = len(dates)-1
    found_sdate = False
    while sindex < tindex and (not found_sdate):
        mindex = int((sindex + tindex)/2)
        tmp = get_bar_data_v5(category, symbol, interval, dt_to_millsec(dates[mindex]))
        if tmp is None or (len(tmp) == 0):
            sindex = mindex + 1
        elif len(tmp) >= 200:
            tindex = mindex
        else:
            found_sdate = True
    if not found_sdate:
        return None
    else:
        return dates[mindex]

In [13]:
def query_histdata_v5(category, symbol, interval, sdate=None):
    # interval param: 1 3 5 15 30 60 120 240 360 720 "D" "M" "W"
    if interval in ['D', 'W', 'M']:
        interval0 = 86400
    else:
        interval0 = interval
    if sdate is None:
        if category == 'spot':
            sdate = find_sdate(category, symbol, interval)
        else:
            sdate = '2020-01-01'
    if sdate is None:
        return pd.DataFrame()
    starttime = dt.datetime.strptime(sdate, '%Y-%m-%d')
    df_list = []
    last_second = dt_to_millsec(sdate)
    while True:
        print(dt.datetime.fromtimestamp((int(last_second/1000))))
        new_df = get_bar_data_v5(category, symbol, interval, last_second)
        if new_df is None or int(new_df['startTime'].max()) < last_second:
            break
        df_list.append(new_df)
        last_second = int(new_df['startTime'].max()) + interval0 * 1000
    df = pd.concat(df_list)
    return df

In [8]:
def query_histdata(symbol, interval0, sdate, exchange='bybit'):
    # interval param: 1 3 5 15 30 60 120 240 360 720 "D" "M" "W"
    if interval0 == 'D':
        interval = 1440
    elif interval0 == 'W':
        interval = 10080
    elif interval0 == 'M':
        interval = 44640
    else:
        interval = interval0
    starttime = dt.datetime.strptime(sdate, '%Y-%m-%d')
    df_list = []
    last_datetime = str(int(starttime.timestamp()))
    while True:
        print(dt.datetime.fromtimestamp((int(last_datetime))))
        new_df = get_bar_data(symbol, interval, last_datetime, exchange=exchange)
        if new_df is None:
            break
        df_list.append(new_df)
        if exchange=='bybit':
            print(dt.datetime.fromtimestamp(new_df['open_time'].max()))
            last_datetime = new_df['open_time'].max() + 60 * interval
        else:
            last_datetime = new_df['start_at'].max() + 60 * interval
    df = pd.concat(df_list)
    return df

In [9]:
def query_orderbook_futures(symbol):
    url = "https://api.bybit.com/v2/public/orderBook/L2"
    return pd.DataFrame(json.loads(requests.get(url, params = {"symbol" : symbol}).text)['result'])

In [10]:
def query_public_trading_history(category, symbol):
    url = "https://api.bybit.com/v5/market/recent-trade"
    df = pd.DataFrame(list(json.loads(requests.get(url, params = {"category": category, "symbol" : symbol}).text)['result']['list']))
    df.index = [dt.datetime.fromtimestamp(int(x)/1000) for x in df.time]
    df = df.iloc[::-1]
    return df

In [11]:
# not finished yet
def query_open_Interest(symbol, intervalTime, startTime):
    # intervalTime: 5min,15min,30min,1h,4h,1d
    url = "https://api.bybit.com/v5/market/open-interest"
    req_params = {"symbol" : symbol, 'interval' : interval, 'from' : startTime}
    return pd.DataFrame(json.loads(requests.get(url, params = {"symbol" : symbol}).text)['result'])

In [3]:
def query_basicinfo_futures():
    url = "https://api.bybit.com/v2/public/symbols"
    basicinfo = pd.DataFrame(json.loads(requests.get(url).text)['result'])[['name', 'alias', 'status', 'base_currency', 'quote_currency', 'taker_fee', 'maker_fee']]
    return basicinfo

In [ ]:
def query_basicinfo_spot():
    url = "https://api.bybit.com/spot/v1/symbols"
    basicinfo = pd.DataFrame(json.loads(requests.get(url).text)['result'])[['name', 'alias', 'baseCurrency', 'quoteCurrency', 'minPricePrecision']]
    return basicinfo

In [5]:
basicinfo_spot = query_basicinfo_spot()

In [8]:
basicinfo_spot

,name,alias,baseCurrency,quoteCurrency,minPricePrecision
0,BTCUSDT,BTCUSDT,BTC,USDT,0.01
1,ETHUSDT,ETHUSDT,ETH,USDT,0.01
2,XRPUSDT,XRPUSDT,XRP,USDT,0.0001
3,EOSUSDT,EOSUSDT,EOS,USDT,0.0001
4,ETHBTC,ETHBTC,ETH,BTC,0.000001
...,...,...,...,...,...
371,AXLUSDT,AXLUSDT,AXL,USDT,0.0001
372,CGPTUSDT,CGPTUSDT,CGPT,USDT,0.000001
373,PLAYUSDT,PLAYUSDT,PLAY,USDT,0.00001
374,AGIUSDT,AGIUSDT,AGI,USDT,0.00001


In [6]:
basicinfo = query_basicinfo_futures()

In [7]:
basicinfo

,name,alias,status,base_currency,quote_currency,taker_fee,maker_fee
0,10000NFTUSDT,10000NFTUSDT,Trading,10000NFT,USDT,0.0006,0.0001
1,1000BONKUSDT,1000BONKUSDT,Trading,1000BONK,USDT,0.0006,0.0001
2,1000BTTUSDT,1000BTTUSDT,Trading,1000BTT,USDT,0.0006,0.0001
3,1000FLOKIUSDT,1000FLOKIUSDT,Trading,1000FLOKI,USDT,0.0006,0.0001
4,1000LUNCUSDT,1000LUNCUSDT,Trading,1000LUNC,USDT,0.0006,0.0001
...,...,...,...,...,...,...,...
217,YGGUSDT,YGGUSDT,Trading,YGG,USDT,0.0006,0.0001
218,ZECUSDT,ZECUSDT,Trading,ZEC,USDT,0.0006,0.0001
219,ZENUSDT,ZENUSDT,Trading,ZEN,USDT,0.0006,0.0001
220,ZILUSDT,ZILUSDT,Trading,ZIL,USDT,0.0006,0.0001


In [16]:
def download_crypto_data(saving_folder, sdate, interval, sindex=0, tindex=len(basicinfo), source='bybit', returndict=False, assettype='futures'):
    datadict = {}
    for i in range(sindex, tindex):
        if basicinfo['quote_currency'].iloc[i]!='USDT':
            continue
        print(basicinfo['alias'].iloc[i])
        try:
            datadict[basicinfo['alias'].iloc[i]] = query_histdata(basicinfo['alias'].iloc[i], interval, sdate, source)
            datadict[basicinfo['alias'].iloc[i]].to_csv(saving_folder + basicinfo['alias'].iloc[i]+'.csv')
        except Exception as e:
            print(e)
    if returndict:
        return datadict

In [21]:
def download_crypto_data_v5(basicinfo, assettype, saving_folder, sdate, interval, sindex=0, tindex=None, returndict=False):
    datadict = {}
    if assettype == 'futures':
        assettype = 'linear'
    if tindex is None:
        tindex = len(basicinfo)
    if 'quoteCurrency' in basicinfo.columns:
        quotecol = 'quoteCurrency'
    else:
        quotecol = 'quote_currency'
    for i in range(sindex, tindex):
        if basicinfo[quotecol].iloc[i]!='USDT':
            continue
        print(basicinfo['alias'].iloc[i])
        try:
            datadict[basicinfo['alias'].iloc[i]] = query_histdata_v5(assettype, basicinfo['alias'].iloc[i], interval, sdate=None)
            datadict[basicinfo['alias'].iloc[i]].to_csv(saving_folder + basicinfo['alias'].iloc[i]+'.csv')
        except Exception as e:
            print(e)
    if returndict:
        return datadict

In [18]:
# daily data
download_crypto_data_v5(basicinfo=basicinfo_spot,
                        assettype='spot',
                        saving_folder='C:\\Users\\paperspace\\Documents\\wmlakefund\\data\\v5\\spot\\daily\\', 
                        sdate='2020-01-01', 
                        interval='D', 
                        sindex=0,
                        tindex=len(basicinfo_spot),
                        returndict=False)

BTCUSDT
2021-04-05 00:00:00
2021-10-23 00:00:00
2022-05-12 00:00:00
2022-11-29 00:00:00
2023-04-21 00:00:00
ETHUSDT
2021-04-05 00:00:00
2021-10-23 00:00:00
2022-05-12 00:00:00
2022-11-29 00:00:00
2023-04-21 00:00:00
XRPUSDT
2021-04-05 00:00:00
2021-10-23 00:00:00
2022-05-12 00:00:00
2022-11-29 00:00:00
2023-04-21 00:00:00
EOSUSDT
2021-04-05 00:00:00
2021-10-23 00:00:00
2022-05-12 00:00:00
2022-11-29 00:00:00
2023-04-21 00:00:00
DOTUSDT
2021-04-05 00:00:00
2021-10-23 00:00:00
2022-05-12 00:00:00
2022-11-29 00:00:00
2023-04-21 00:00:00
XLMUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
LTCUSDT
2021-04-05 00:00:00
2021-10-23 00:00:00
2022-05-12 00:00:00
2022-11-29 00:00:00
2023-04-21 00:00:00
DOGEUSDT
2021-04-05 00:00:00
2021-10-23 00:00:00
2022-05-12 00:00:00
2022-11-29 00:00:00
2023-04-21 00:00:00
BITUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
CHZUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00

2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
LRCUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
LDOUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
REALUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
KRLUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
DEVTUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
CRAFTUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
1SOLUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
PLTUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
IZIUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
QTUMUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-10-08 00:00:00
2023-04-21 00:00:00
DCRUSDT
2021-09-01 00:00:00
2022-03-21 00:00:00
2022-1

FITFIUSDT
2022-01-28 00:00:00
2022-08-17 00:00:00
2023-03-06 00:00:00
2023-04-21 00:00:00
STRMUSDT
2022-01-28 00:00:00
2022-08-17 00:00:00
2023-03-06 00:00:00
2023-04-21 00:00:00
GALUSDT
2022-01-28 00:00:00
2022-08-17 00:00:00
2023-03-06 00:00:00
2023-04-21 00:00:00
ETH3SUSDT
2022-01-28 00:00:00
2022-08-17 00:00:00
2023-03-06 00:00:00
2023-04-21 00:00:00
ETH3LUSDT
2022-01-28 00:00:00
2022-08-17 00:00:00
2023-03-06 00:00:00
2023-04-21 00:00:00
KOKUSDT
2022-01-28 00:00:00
2022-08-17 00:00:00
2023-03-06 00:00:00
2023-04-21 00:00:00
FAMEUSDT
2022-01-28 00:00:00
2022-08-17 00:00:00
2023-03-06 00:00:00
2023-04-21 00:00:00
XRP3SUSDT
2022-01-28 00:00:00
2022-08-17 00:00:00
2023-03-06 00:00:00
2023-04-21 00:00:00
XRP3LUSDT
2022-01-28 00:00:00
2022-08-17 00:00:00
2023-03-06 00:00:00
2023-04-21 00:00:00
USDDUSDT
2022-01-28 00:00:00
2022-08-17 00:00:00
2023-03-06 00:00:00
2023-04-21 00:00:00
OPUSDT
2022-01-28 00:00:00
2022-08-17 00:00:00
2023-03-06 00:00:00
2023-04-21 00:00:00
LUNAUSDT
2022-01-28 

MEEUSDT
2022-06-26 00:00:00
2023-01-13 00:00:00
2023-04-21 00:00:00
TONUSDT
2022-06-26 00:00:00
2023-01-13 00:00:00
2023-04-21 00:00:00
BONKUSDT
2022-06-26 00:00:00
2023-01-13 00:00:00
2023-04-21 00:00:00
FLRUSDT
2022-06-26 00:00:00
2023-01-13 00:00:00
2023-04-21 00:00:00
TIMEUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
3PUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
RPLUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
SSVUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
FXSUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
COREUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
RDNTUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
BLURUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
LISUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
AGIXUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
MDAOUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
ACSUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
HVHUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
GNSUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
DPXUSDT
2022-11-22 00:00:00
2023-04-21 00:00:00
PI

In [19]:
# hourly data
download_crypto_data_v5(basicinfo=basicinfo_spot,
                        assettype='spot',
                        saving_folder='C:\\Users\\paperspace\\Documents\\wmlakefund\\data\\v5\\spot\\hourly\\', 
                        sdate='2020-01-01', 
                        interval=60, 
                        sindex=0,
                        tindex=len(basicinfo_spot),
                        returndict=False)

BTCUSDT
2021-06-29 00:00:00
2021-07-07 08:01:00
2021-07-15 16:01:00
2021-07-24 00:01:00
2021-08-01 08:01:00
2021-08-09 16:01:00
2021-08-18 00:01:00
2021-08-26 08:01:00
2021-09-03 16:01:00
2021-09-12 00:01:00
2021-09-20 08:01:00
2021-09-28 16:01:00
2021-10-07 00:01:00
2021-10-15 08:01:00
2021-10-23 16:01:00
2021-11-01 00:01:00
2021-11-09 08:01:00
2021-11-17 16:01:00
2021-11-26 00:01:00
2021-12-04 08:01:00
2021-12-12 16:01:00
2021-12-21 00:01:00
2021-12-29 08:01:00
2022-01-06 16:01:00
2022-01-15 00:01:00
2022-01-23 08:01:00
2022-01-31 16:01:00
2022-02-09 00:01:00
2022-02-17 08:01:00
2022-02-25 16:01:00
2022-03-06 00:01:00
2022-03-14 08:01:00
2022-03-22 16:01:00
2022-03-31 00:01:00
2022-04-08 08:01:00
2022-04-16 16:01:00
2022-04-25 00:01:00
2022-05-03 08:01:00
2022-05-11 16:01:00
2022-05-20 00:01:00
2022-05-28 08:01:00
2022-06-05 16:01:00
2022-06-14 00:01:00
2022-06-22 08:01:00
2022-06-30 16:01:00
2022-07-09 00:01:00
2022-07-17 08:01:00
2022-07-25 16:01:00
2022-08-03 00:01:00
2022-08-11 0

2021-12-26 16:01:00
2022-01-04 00:01:00
2022-01-12 08:01:00
2022-01-20 16:01:00
2022-01-29 00:01:00
2022-02-06 08:01:00
2022-02-14 16:01:00
2022-02-23 00:01:00
2022-03-03 08:01:00
2022-03-11 16:01:00
2022-03-20 00:01:00
2022-03-28 08:01:00
2022-04-05 16:01:00
2022-04-14 00:01:00
2022-04-22 08:01:00
2022-04-30 16:01:00
2022-05-09 00:01:00
2022-05-17 08:01:00
2022-05-25 16:01:00
2022-06-03 00:01:00
2022-06-11 08:01:00
2022-06-19 16:01:00
2022-06-28 00:01:00
2022-07-06 08:01:00
2022-07-14 16:01:00
2022-07-23 00:01:00
2022-07-31 08:01:00
2022-08-08 16:01:00
2022-08-17 00:01:00
2022-08-25 08:01:00
2022-09-02 16:01:00
2022-09-11 00:01:00
2022-09-19 08:01:00
2022-09-27 16:01:00
2022-10-06 00:01:00
2022-10-14 08:01:00
2022-10-22 16:01:00
2022-10-31 00:01:00
2022-11-08 08:01:00
2022-11-16 16:01:00
2022-11-25 00:01:00
2022-12-03 08:01:00
2022-12-11 16:01:00
2022-12-20 00:01:00
2022-12-28 08:01:00
2023-01-05 16:01:00
2023-01-14 00:01:00
2023-01-22 08:01:00
2023-01-30 16:01:00
2023-02-08 00:01:00


2023-02-20 08:01:00
2023-02-28 16:01:00
2023-03-09 00:01:00
2023-03-17 08:01:00
2023-03-25 16:01:00
2023-04-03 00:01:00
2023-04-11 08:01:00
2023-04-19 16:01:00
2023-04-20 06:01:00
MANAUSDT
2021-09-30 00:00:00
2021-10-08 08:01:00
2021-10-16 16:01:00
2021-10-25 00:01:00
2021-11-02 08:01:00
2021-11-10 16:01:00
2021-11-19 00:01:00
2021-11-27 08:01:00
2021-12-05 16:01:00
2021-12-14 00:01:00
2021-12-22 08:01:00
2021-12-30 16:01:00
2022-01-08 00:01:00
2022-01-16 08:01:00
2022-01-24 16:01:00
2022-02-02 00:01:00
2022-02-10 08:01:00
2022-02-18 16:01:00
2022-02-27 00:01:00
2022-03-07 08:01:00
2022-03-15 16:01:00
2022-03-24 00:01:00
2022-04-01 08:01:00
2022-04-09 16:01:00
2022-04-18 00:01:00
2022-04-26 08:01:00
2022-05-04 16:01:00
2022-05-13 00:01:00
2022-05-21 08:01:00
2022-05-29 16:01:00
2022-06-07 00:01:00
2022-06-15 08:01:00
2022-06-23 16:01:00
2022-07-02 00:01:00
2022-07-10 08:01:00
2022-07-18 16:01:00
2022-07-27 00:01:00
2022-08-04 08:01:00
2022-08-12 16:01:00
2022-08-21 00:01:00
2022-08-29 

2022-09-30 00:01:00
2022-10-08 08:01:00
2022-10-16 16:01:00
2022-10-25 00:01:00
2022-11-02 08:01:00
2022-11-10 16:01:00
2022-11-19 00:01:00
2022-11-27 08:01:00
2022-12-05 16:01:00
2022-12-14 00:01:00
2022-12-22 08:01:00
2022-12-30 16:01:00
2023-01-08 00:01:00
2023-01-16 08:01:00
2023-01-24 16:01:00
2023-02-02 00:01:00
2023-02-10 08:01:00
2023-02-18 16:01:00
2023-02-27 00:01:00
2023-03-07 08:01:00
2023-03-15 16:01:00
2023-03-24 00:01:00
2023-04-01 08:01:00
2023-04-09 16:01:00
2023-04-18 00:01:00
2023-04-20 07:01:00
LINKUSDT
2021-09-20 00:00:00
2021-09-28 08:01:00
2021-10-06 16:01:00
2021-10-15 00:01:00
2021-10-23 08:01:00
2021-10-31 16:01:00
2021-11-09 00:01:00
2021-11-17 08:01:00
2021-11-25 16:01:00
2021-12-04 00:01:00
2021-12-12 08:01:00
2021-12-20 16:01:00
2021-12-29 00:01:00
2022-01-06 08:01:00
2022-01-14 16:01:00
2022-01-23 00:01:00
2022-01-31 08:01:00
2022-02-08 16:01:00
2022-02-17 00:01:00
2022-02-25 08:01:00
2022-03-05 16:01:00
2022-03-14 00:01:00
2022-03-22 08:01:00
2022-03-30 

2022-07-21 00:01:00
2022-07-29 08:01:00
2022-08-06 16:01:00
2022-08-15 00:01:00
2022-08-23 08:01:00
2022-08-31 16:01:00
2022-09-09 00:01:00
2022-09-17 08:01:00
2022-09-25 16:01:00
2022-10-04 00:01:00
2022-10-12 08:01:00
2022-10-20 16:01:00
2022-10-29 00:01:00
2022-11-06 08:01:00
2022-11-14 16:01:00
2022-11-23 00:01:00
2022-12-01 08:01:00
2022-12-09 16:01:00
2022-12-18 00:01:00
2022-12-26 08:01:00
2023-01-03 16:01:00
2023-01-12 00:01:00
2023-01-20 08:01:00
2023-01-28 16:01:00
2023-02-06 00:01:00
2023-02-14 08:01:00
2023-02-22 16:01:00
2023-03-03 00:01:00
2023-03-11 08:01:00
2023-03-19 16:01:00
2023-03-28 00:01:00
2023-04-05 08:01:00
2023-04-13 16:01:00
2023-04-20 07:01:00
ETCUSDT
2022-07-25 00:00:00
2022-08-02 08:01:00
2022-08-10 16:01:00
2022-08-19 00:01:00
2022-08-27 08:01:00
2022-09-04 16:01:00
2022-09-13 00:01:00
2022-09-21 08:01:00
2022-09-29 16:01:00
2022-10-08 00:01:00
2022-10-16 08:01:00
2022-10-24 16:01:00
2022-11-02 00:01:00
2022-11-10 08:01:00
2022-11-18 16:01:00
2022-11-27 0

...

2022-11-27 16:01:00
2022-12-06 00:01:00
2022-12-14 08:01:00
2022-12-22 16:01:00
2022-12-31 00:01:00
2023-01-08 08:01:00
2023-01-16 16:01:00
2023-01-25 00:01:00
2023-02-02 08:01:00
2023-02-10 16:01:00
2023-02-19 00:01:00
2023-02-27 08:01:00
2023-03-07 16:01:00
2023-03-16 00:01:00
2023-03-24 08:01:00
2023-04-01 16:01:00
2023-04-10 00:01:00
2023-04-18 08:01:00
2023-04-20 09:01:00
GMT2LUSDT
2022-08-03 00:00:00
2022-08-11 08:01:00
2022-08-19 16:01:00
2022-08-28 00:01:00
2022-09-05 08:01:00
2022-09-13 16:01:00
2022-09-22 00:01:00
2022-09-30 08:01:00
2022-10-08 16:01:00
2022-10-17 00:01:00
2022-10-25 08:01:00
2022-11-02 16:01:00
2022-11-11 00:01:00
2022-11-19 08:01:00
2022-11-27 16:01:00
2022-12-06 00:01:00
2022-12-14 08:01:00
2022-12-22 16:01:00
2022-12-31 00:01:00
2023-01-08 08:01:00
2023-01-16 16:01:00
2023-01-25 00:01:00
2023-02-02 08:01:00
2023-02-10 16:01:00
2023-02-19 00:01:00
2023-02-27 08:01:00
2023-03-07 16:01:00
2023-03-16 00:01:00
2023-03-24 08:01:00
2023-04-01 16:01:00
2023-04-10

2022-08-13 00:00:00
2022-08-21 08:01:00
2022-08-29 16:01:00
2022-09-07 00:01:00
2022-09-15 08:01:00
2022-09-23 16:01:00
2022-10-02 00:01:00
2022-10-10 08:01:00
2022-10-18 16:01:00
2022-10-27 00:01:00
2022-11-04 08:01:00
2022-11-12 16:01:00
2022-11-21 00:01:00
2022-11-29 08:01:00
2022-12-07 16:01:00
2022-12-16 00:01:00
2022-12-24 08:01:00
2023-01-01 16:01:00
2023-01-10 00:01:00
2023-01-18 08:01:00
2023-01-26 16:01:00
2023-02-04 00:01:00
2023-02-12 08:01:00
2023-02-20 16:01:00
2023-03-01 00:01:00
2023-03-09 08:01:00
2023-03-17 16:01:00
2023-03-26 00:01:00
2023-04-03 08:01:00
2023-04-11 16:01:00
2023-04-20 00:01:00
2023-04-20 09:01:00
SOLOUSDT
2022-08-13 00:00:00
2022-08-21 08:01:00
2022-08-29 16:01:00
2022-09-07 00:01:00
2022-09-15 08:01:00
2022-09-23 16:01:00
2022-10-02 00:01:00
2022-10-10 08:01:00
2022-10-18 16:01:00
2022-10-27 00:01:00
2022-11-04 08:01:00
2022-11-12 16:01:00
2022-11-21 00:01:00
2022-11-29 08:01:00
2022-12-07 16:01:00
2022-12-16 00:01:00
2022-12-24 08:01:00
2023-01-01 

2023-02-22 16:01:00
2023-03-03 00:01:00
2023-03-11 08:01:00
2023-03-19 16:01:00
2023-03-28 00:01:00
2023-04-05 08:01:00
2023-04-13 16:01:00
2023-04-20 09:01:00
OKGUSDT
2022-09-09 00:00:00
2022-09-17 08:01:00
2022-09-25 16:01:00
2022-10-04 00:01:00
2022-10-12 08:01:00
2022-10-20 16:01:00
2022-10-29 00:01:00
2022-11-06 08:01:00
2022-11-14 16:01:00
2022-11-23 00:01:00
2022-12-01 08:01:00
2022-12-09 16:01:00
2022-12-18 00:01:00
2022-12-26 08:01:00
2023-01-03 16:01:00
2023-01-12 00:01:00
2023-01-20 08:01:00
2023-01-28 16:01:00
2023-02-06 00:01:00
2023-02-14 08:01:00
2023-02-22 16:01:00
2023-03-03 00:01:00
2023-03-11 08:01:00
2023-03-19 16:01:00
2023-03-28 00:01:00
2023-04-05 08:01:00
2023-04-13 16:01:00
2023-04-20 09:01:00
ETHWUSDT
2022-09-09 00:00:00
2022-09-17 08:01:00
2022-09-25 16:01:00
2022-10-04 00:01:00
2022-10-12 08:01:00
2022-10-20 16:01:00
2022-10-29 00:01:00
2022-11-06 08:01:00
2022-11-14 16:01:00
2022-11-23 00:01:00
2022-12-01 08:01:00
2022-12-09 16:01:00
2022-12-18 00:01:00
202

2023-01-26 08:01:00
2023-02-03 16:01:00
2023-02-12 00:01:00
2023-02-20 08:01:00
2023-02-28 16:01:00
2023-03-09 00:01:00
2023-03-17 08:01:00
2023-03-25 16:01:00
2023-04-03 00:01:00
2023-04-11 08:01:00
2023-04-19 16:01:00
2023-04-20 09:01:00
HFTUSDT
2022-11-04 00:00:00
2022-11-12 08:01:00
2022-11-20 16:01:00
2022-11-29 00:01:00
2022-12-07 08:01:00
2022-12-15 16:01:00
2022-12-24 00:01:00
2023-01-01 08:01:00
2023-01-09 16:01:00
2023-01-18 00:01:00
2023-01-26 08:01:00
2023-02-03 16:01:00
2023-02-12 00:01:00
2023-02-20 08:01:00
2023-02-28 16:01:00
2023-03-09 00:01:00
2023-03-17 08:01:00
2023-03-25 16:01:00
2023-04-03 00:01:00
2023-04-11 08:01:00
2023-04-19 16:01:00
2023-04-20 09:01:00
KCALUSDT
2022-11-04 00:00:00
2022-11-12 08:01:00
2022-11-20 16:01:00
2022-11-29 00:01:00
2022-12-07 08:01:00
2022-12-15 16:01:00
2022-12-24 00:01:00
2023-01-01 08:01:00
2023-01-09 16:01:00
2023-01-18 00:01:00
2023-01-26 08:01:00
2023-02-03 16:01:00
2023-02-12 00:01:00
2023-02-20 08:01:00
2023-02-28 16:01:00
202

2023-04-06 00:01:00
2023-04-14 08:01:00
2023-04-20 09:01:00
HVHUSDT
2023-02-15 00:00:00
2023-02-23 08:01:00
2023-03-03 16:01:00
2023-03-12 00:01:00
2023-03-20 08:01:00
2023-03-28 16:01:00
2023-04-06 00:01:00
2023-04-14 08:01:00
2023-04-20 09:01:00
GNSUSDT
2023-02-15 00:00:00
2023-02-23 08:01:00
2023-03-03 16:01:00
2023-03-12 00:01:00
2023-03-20 08:01:00
2023-03-28 16:01:00
2023-04-06 00:01:00
2023-04-14 08:01:00
2023-04-20 09:01:00
DPXUSDT
2023-02-24 00:00:00
2023-03-04 08:01:00
2023-03-12 16:01:00
2023-03-21 00:01:00
2023-03-29 08:01:00
2023-04-06 16:01:00
2023-04-15 00:01:00
2023-04-20 09:01:00
PIPUSDT
2023-02-24 00:00:00
2023-03-04 08:01:00
2023-03-12 16:01:00
2023-03-21 00:01:00
2023-03-29 08:01:00
2023-04-06 16:01:00
2023-04-15 00:01:00
2023-04-20 09:01:00
PRIMEUSDT
2023-02-24 00:00:00
2023-03-04 08:01:00
2023-03-12 16:01:00
2023-03-21 00:01:00
2023-03-29 08:01:00
2023-04-06 16:01:00
2023-04-15 00:01:00
2023-04-20 09:01:00
EVERUSDT
2023-02-24 00:00:00
2023-03-04 08:01:00
2023-03-1

In [20]:
# minute bar data
download_crypto_data_v5(basicinfo=basicinfo_spot,
                        assettype='spot',
                        saving_folder='C:\\Users\\paperspace\\Documents\\wmlakefund\\data\\v5\\spot\\minute_bar\\', 
                        sdate='2020-01-01', 
                        interval=1, 
                        sindex=0,
                        tindex=len(basicinfo_spot),
                        returndict=False)

BTCUSDT
ETHUSDT
XRPUSDT
2021-07-20 00:00:00
2021-07-20 03:20:01
2021-07-20 06:40:01
2021-07-20 10:00:01
2021-07-20 13:20:01
2021-07-20 16:40:01
2021-07-20 20:00:01
2021-07-20 23:20:01
2021-07-21 02:40:01
2021-07-21 06:00:01
2021-07-21 09:20:01
2021-07-21 12:40:01
2021-07-21 16:00:01
2021-07-21 19:20:01
2021-07-21 22:40:01
2021-07-22 02:00:01
2021-07-22 05:20:01
2021-07-22 08:40:01
2021-07-22 12:00:01
2021-07-22 15:20:01
2021-07-22 18:40:01
2021-07-22 22:00:01
2021-07-23 01:20:01
2021-07-23 04:40:01
2021-07-23 08:00:01
2021-07-23 11:20:01
2021-07-23 14:40:01
2021-07-23 18:00:01
2021-07-23 21:20:01
2021-07-24 00:40:01
2021-07-24 04:00:01
2021-07-24 07:20:01
2021-07-24 10:40:01
2021-07-24 14:00:01
2021-07-24 17:20:01
2021-07-24 20:40:01
2021-07-25 00:00:01
2021-07-25 03:20:01
2021-07-25 06:40:01
2021-07-25 10:00:01
2021-07-25 13:20:01
2021-07-25 16:40:01
2021-07-25 20:00:01
2021-07-25 23:20:01
2021-07-26 02:40:01
2021-07-26 06:00:01
2021-07-26 09:20:01
2021-07-26 12:40:01
2021-07-26 16:00

2021-09-14 19:20:01
2021-09-14 22:40:01
2021-09-15 02:00:01
2021-09-15 05:20:01
2021-09-15 08:40:01
2021-09-15 12:00:01
2021-09-15 15:20:01
2021-09-15 18:40:01
2021-09-15 22:00:01
2021-09-16 01:20:01
2021-09-16 04:40:01
2021-09-16 08:00:01
2021-09-16 11:20:01
2021-09-16 14:40:01
2021-09-16 18:00:01
2021-09-16 21:20:01
2021-09-17 00:40:01
2021-09-17 04:00:01
2021-09-17 07:20:01
2021-09-17 10:40:01
2021-09-17 14:00:01
2021-09-17 17:20:01
2021-09-17 20:40:01
2021-09-18 00:00:01
2021-09-18 03:20:01
2021-09-18 06:40:01
2021-09-18 10:00:01
2021-09-18 13:20:01
2021-09-18 16:40:01
2021-09-18 20:00:01
2021-09-18 23:20:01
2021-09-19 02:40:01
2021-09-19 06:00:01
2021-09-19 09:20:01
2021-09-19 12:40:01
2021-09-19 16:00:01
2021-09-19 19:20:01
2021-09-19 22:40:01
2021-09-20 02:00:01
2021-09-20 05:20:01
2021-09-20 08:40:01
2021-09-20 12:00:01
2021-09-20 15:20:01
2021-09-20 18:40:01
2021-09-20 22:00:01
2021-09-21 01:20:01
2021-09-21 04:40:01
2021-09-21 08:00:01
2021-09-21 11:20:01
2021-09-21 14:40:01


2021-11-10 18:00:01
2021-11-10 21:20:01
2021-11-11 00:40:01
2021-11-11 04:00:01
2021-11-11 07:20:01
2021-11-11 10:40:01
2021-11-11 14:00:01
2021-11-11 17:20:01
2021-11-11 20:40:01
2021-11-12 00:00:01
2021-11-12 03:20:01
2021-11-12 06:40:01
2021-11-12 10:00:01
2021-11-12 13:20:01
2021-11-12 16:40:01
2021-11-12 20:00:01
2021-11-12 23:20:01
2021-11-13 02:40:01
2021-11-13 06:00:01
2021-11-13 09:20:01
2021-11-13 12:40:01
2021-11-13 16:00:01
2021-11-13 19:20:01
2021-11-13 22:40:01
2021-11-14 02:00:01
2021-11-14 05:20:01
2021-11-14 08:40:01
2021-11-14 12:00:01
2021-11-14 15:20:01
2021-11-14 18:40:01
2021-11-14 22:00:01
2021-11-15 01:20:01
2021-11-15 04:40:01
2021-11-15 08:00:01
2021-11-15 11:20:01
2021-11-15 14:40:01
2021-11-15 18:00:01
2021-11-15 21:20:01
2021-11-16 00:40:01
2021-11-16 04:00:01
2021-11-16 07:20:01
2021-11-16 10:40:01
2021-11-16 14:00:01
2021-11-16 17:20:01
2021-11-16 20:40:01
2021-11-17 00:00:01
2021-11-17 03:20:01
2021-11-17 06:40:01
2021-11-17 10:00:01
2021-11-17 13:20:01


2022-01-06 16:40:01
2022-01-06 20:00:01
2022-01-06 23:20:01
2022-01-07 02:40:01
2022-01-07 06:00:01
2022-01-07 09:20:01
2022-01-07 12:40:01
2022-01-07 16:00:01
2022-01-07 19:20:01
2022-01-07 22:40:01
2022-01-08 02:00:01
2022-01-08 05:20:01
2022-01-08 08:40:01
2022-01-08 12:00:01
2022-01-08 15:20:01
2022-01-08 18:40:01
2022-01-08 22:00:01
2022-01-09 01:20:01
2022-01-09 04:40:01
2022-01-09 08:00:01
2022-01-09 11:20:01
2022-01-09 14:40:01
2022-01-09 18:00:01
2022-01-09 21:20:01
2022-01-10 00:40:01
2022-01-10 04:00:01
2022-01-10 07:20:01
2022-01-10 10:40:01
2022-01-10 14:00:01
2022-01-10 17:20:01
2022-01-10 20:40:01
2022-01-11 00:00:01
2022-01-11 03:20:01
2022-01-11 06:40:01
2022-01-11 10:00:01
2022-01-11 13:20:01
2022-01-11 16:40:01
2022-01-11 20:00:01
2022-01-11 23:20:01
2022-01-12 02:40:01
2022-01-12 06:00:01
2022-01-12 09:20:01
2022-01-12 12:40:01
2022-01-12 16:00:01
2022-01-12 19:20:01
2022-01-12 22:40:01
2022-01-13 02:00:01
2022-01-13 05:20:01
2022-01-13 08:40:01
2022-01-13 12:00:01


2022-03-04 15:20:01
2022-03-04 18:40:01
2022-03-04 22:00:01
2022-03-05 01:20:01
2022-03-05 04:40:01
2022-03-05 08:00:01
2022-03-05 11:20:01
2022-03-05 14:40:01
2022-03-05 18:00:01
2022-03-05 21:20:01
2022-03-06 00:40:01
2022-03-06 04:00:01
2022-03-06 07:20:01
2022-03-06 10:40:01
2022-03-06 14:00:01
2022-03-06 17:20:01
2022-03-06 20:40:01
2022-03-07 00:00:01
2022-03-07 03:20:01
2022-03-07 06:40:01
2022-03-07 10:00:01
2022-03-07 13:20:01
2022-03-07 16:40:01
2022-03-07 20:00:01
2022-03-07 23:20:01
2022-03-08 02:40:01
2022-03-08 06:00:01
2022-03-08 09:20:01
2022-03-08 12:40:01
2022-03-08 16:00:01
2022-03-08 19:20:01
2022-03-08 22:40:01
2022-03-09 02:00:01
2022-03-09 05:20:01
2022-03-09 08:40:01
2022-03-09 12:00:01
2022-03-09 15:20:01
2022-03-09 18:40:01
2022-03-09 22:00:01
2022-03-10 01:20:01
2022-03-10 04:40:01
2022-03-10 08:00:01
2022-03-10 11:20:01
2022-03-10 14:40:01
2022-03-10 18:00:01
2022-03-10 21:20:01
2022-03-11 00:40:01
2022-03-11 04:00:01
2022-03-11 07:20:01
2022-03-11 10:40:01


...

2023-01-15 01:20:01
2023-01-15 04:40:01
2023-01-15 08:00:01
2023-01-15 11:20:01
2023-01-15 14:40:01
2023-01-15 18:00:01
2023-01-15 21:20:01
2023-01-16 00:40:01
2023-01-16 04:00:01
2023-01-16 07:20:01
2023-01-16 10:40:01
2023-01-16 14:00:01
2023-01-16 17:20:01
2023-01-16 20:40:01
2023-01-17 00:00:01
2023-01-17 03:20:01
2023-01-17 06:40:01
2023-01-17 10:00:01
2023-01-17 13:20:01
2023-01-17 16:40:01
2023-01-17 20:00:01
2023-01-17 23:20:01
2023-01-18 02:40:01
2023-01-18 06:00:01
2023-01-18 09:20:01
2023-01-18 12:40:01
2023-01-18 16:00:01
2023-01-18 19:20:01
2023-01-18 22:40:01
2023-01-19 02:00:01
2023-01-19 05:20:01
2023-01-19 08:40:01
2023-01-19 12:00:01
2023-01-19 15:20:01
2023-01-19 18:40:01
2023-01-19 22:00:01
2023-01-20 01:20:01
2023-01-20 04:40:01
2023-01-20 08:00:01
2023-01-20 11:20:01
2023-01-20 14:40:01
2023-01-20 18:00:01
2023-01-20 21:20:01
2023-01-21 00:40:01
2023-01-21 04:00:01
2023-01-21 07:20:01
2023-01-21 10:40:01
2023-01-21 14:00:01
2023-01-21 17:20:01
2023-01-21 20:40:01


2023-03-13 00:00:01
2023-03-13 03:20:01
2023-03-13 06:40:01
2023-03-13 10:00:01
2023-03-13 13:20:01
2023-03-13 16:40:01
2023-03-13 20:00:01
2023-03-13 23:20:01
2023-03-14 02:40:01
2023-03-14 06:00:01
2023-03-14 09:20:01
2023-03-14 12:40:01
2023-03-14 16:00:01
2023-03-14 19:20:01
2023-03-14 22:40:01
2023-03-15 02:00:01
2023-03-15 05:20:01
2023-03-15 08:40:01
2023-03-15 12:00:01
2023-03-15 15:20:01
2023-03-15 18:40:01
2023-03-15 22:00:01
2023-03-16 01:20:01
2023-03-16 04:40:01
2023-03-16 08:00:01
2023-03-16 11:20:01
2023-03-16 14:40:01
2023-03-16 18:00:01
2023-03-16 21:20:01
2023-03-17 00:40:01
2023-03-17 04:00:01
2023-03-17 07:20:01
2023-03-17 10:40:01
2023-03-17 14:00:01
2023-03-17 17:20:01
2023-03-17 20:40:01
2023-03-18 00:00:01
2023-03-18 03:20:01
2023-03-18 06:40:01
2023-03-18 10:00:01
2023-03-18 13:20:01
2023-03-18 16:40:01
2023-03-18 20:00:01
2023-03-18 23:20:01
2023-03-19 02:40:01
2023-03-19 06:00:01
2023-03-19 09:20:01
2023-03-19 12:40:01
2023-03-19 16:00:01
2023-03-19 19:20:01


2022-11-04 16:00:01
2022-11-04 19:20:01
2022-11-04 22:40:01
2022-11-05 02:00:01
2022-11-05 05:20:01
2022-11-05 08:40:01
2022-11-05 12:00:01
2022-11-05 15:20:01
2022-11-05 18:40:01
2022-11-05 22:00:01
2022-11-06 01:20:01
2022-11-06 04:40:01
2022-11-06 08:00:01
2022-11-06 11:20:01
2022-11-06 14:40:01
2022-11-06 18:00:01
2022-11-06 21:20:01
2022-11-07 00:40:01
2022-11-07 04:00:01
2022-11-07 07:20:01
2022-11-07 10:40:01
2022-11-07 14:00:01
2022-11-07 17:20:01
2022-11-07 20:40:01
2022-11-08 00:00:01
2022-11-08 03:20:01
2022-11-08 06:40:01
2022-11-08 10:00:01
2022-11-08 13:20:01
2022-11-08 16:40:01
2022-11-08 20:00:01
2022-11-08 23:20:01
2022-11-09 02:40:01
2022-11-09 06:00:01
2022-11-09 09:20:01
2022-11-09 12:40:01
2022-11-09 16:00:01
2022-11-09 19:20:01
2022-11-09 22:40:01
2022-11-10 02:00:01
2022-11-10 05:20:01
2022-11-10 08:40:01
2022-11-10 12:00:01
2022-11-10 15:20:01
2022-11-10 18:40:01
2022-11-10 22:00:01
2022-11-11 01:20:01
2022-11-11 04:40:01
2022-11-11 08:00:01
2022-11-11 11:20:01


2022-12-31 14:40:01
2022-12-31 18:00:01
2022-12-31 21:20:01
2023-01-01 00:40:01
2023-01-01 04:00:01
2023-01-01 07:20:01
2023-01-01 10:40:01
2023-01-01 14:00:01
2023-01-01 17:20:01
2023-01-01 20:40:01
2023-01-02 00:00:01
2023-01-02 03:20:01
2023-01-02 06:40:01
2023-01-02 10:00:01
2023-01-02 13:20:01
2023-01-02 16:40:01
2023-01-02 20:00:01
2023-01-02 23:20:01
2023-01-03 02:40:01
2023-01-03 06:00:01
2023-01-03 09:20:01
2023-01-03 12:40:01
2023-01-03 16:00:01
2023-01-03 19:20:01
2023-01-03 22:40:01
2023-01-04 02:00:01
2023-01-04 05:20:01
2023-01-04 08:40:01
2023-01-04 12:00:01
2023-01-04 15:20:01
2023-01-04 18:40:01
2023-01-04 22:00:01
2023-01-05 01:20:01
2023-01-05 04:40:01
2023-01-05 08:00:01
2023-01-05 11:20:01
2023-01-05 14:40:01
2023-01-05 18:00:01
2023-01-05 21:20:01
2023-01-06 00:40:01
2023-01-06 04:00:01
2023-01-06 07:20:01
2023-01-06 10:40:01
2023-01-06 14:00:01
2023-01-06 17:20:01
2023-01-06 20:40:01
2023-01-07 00:00:01
2023-01-07 03:20:01
2023-01-07 06:40:01
2023-01-07 10:00:01


2023-02-26 13:20:01
2023-02-26 16:40:01
2023-02-26 20:00:01
2023-02-26 23:20:01
2023-02-27 02:40:01
2023-02-27 06:00:01
2023-02-27 09:20:01
2023-02-27 12:40:01
2023-02-27 16:00:01
2023-02-27 19:20:01
2023-02-27 22:40:01
2023-02-28 02:00:01
2023-02-28 05:20:01
2023-02-28 08:40:01
2023-02-28 12:00:01
2023-02-28 15:20:01
2023-02-28 18:40:01
2023-02-28 22:00:01
2023-03-01 01:20:01
2023-03-01 04:40:01
2023-03-01 08:00:01
2023-03-01 11:20:01
2023-03-01 14:40:01
2023-03-01 18:00:01
2023-03-01 21:20:01
2023-03-02 00:40:01
2023-03-02 04:00:01
2023-03-02 07:20:01
2023-03-02 10:40:01
2023-03-02 14:00:01
2023-03-02 17:20:01
2023-03-02 20:40:01
2023-03-03 00:00:01
2023-03-03 03:20:01
2023-03-03 06:40:01
2023-03-03 10:00:01
2023-03-03 13:20:01
2023-03-03 16:40:01
2023-03-03 20:00:01
2023-03-03 23:20:01
2023-03-04 02:40:01
2023-03-04 06:00:01
2023-03-04 09:20:01
2023-03-04 12:40:01
2023-03-04 16:00:01
2023-03-04 19:20:01
2023-03-04 22:40:01
2023-03-05 02:00:01
2023-03-05 05:20:01
2023-03-05 08:40:01


In [18]:
basicinfo

,name,alias,status,base_currency,quote_currency,taker_fee,maker_fee
0,10000NFTUSDT,10000NFTUSDT,Trading,10000NFT,USDT,0.0006,0.0001
1,1000BONKUSDT,1000BONKUSDT,Trading,1000BONK,USDT,0.0006,0.0001
2,1000BTTUSDT,1000BTTUSDT,Trading,1000BTT,USDT,0.0006,0.0001
3,1000FLOKIUSDT,1000FLOKIUSDT,Trading,1000FLOKI,USDT,0.0006,0.0001
4,1000LUNCUSDT,1000LUNCUSDT,Trading,1000LUNC,USDT,0.0006,0.0001
...,...,...,...,...,...,...,...
217,YGGUSDT,YGGUSDT,Trading,YGG,USDT,0.0006,0.0001
218,ZECUSDT,ZECUSDT,Trading,ZEC,USDT,0.0006,0.0001
219,ZENUSDT,ZENUSDT,Trading,ZEN,USDT,0.0006,0.0001
220,ZILUSDT,ZILUSDT,Trading,ZIL,USDT,0.0006,0.0001


In [22]:
# daily data
download_crypto_data_v5(basicinfo=basicinfo,
                        assettype='futures',
                        saving_folder='C:\\Users\\paperspace\\Documents\\wmlakefund\\data\\v5\\futures\\daily\\', 
                        sdate='2020-01-01', 
                        interval='D', 
                        sindex=0,
                        tindex=len(basicinfo),
                        returndict=False)

10000NFTUSDT
2020-01-01 00:00:00
2022-08-12 00:00:00
2023-02-28 00:00:00
2023-04-25 00:00:00
1000BONKUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
1000BTTUSDT
2020-01-01 00:00:00
2022-09-13 00:00:00
2023-04-01 00:00:00
2023-04-25 00:00:00
1000FLOKIUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
1000LUNCUSDT
2020-01-01 00:00:00
2023-03-28 00:00:00
2023-04-25 00:00:00
1000XECUSDT
2020-01-01 00:00:00
2022-10-30 00:00:00
2023-04-25 00:00:00
1INCHUSDT
2020-01-01 00:00:00
2022-06-24 00:00:00
2023-01-10 00:00:00
2023-04-25 00:00:00
AAVEUSDT
2020-01-01 00:00:00
2021-11-29 00:00:00
2022-06-17 00:00:00
2023-01-03 00:00:00
2023-04-25 00:00:00
ACHUSDT
2020-01-01 00:00:00
2022-10-25 00:00:00
2023-04-25 00:00:00
ADAUSDT
2020-01-01 00:00:00
2021-10-04 00:00:00
2022-04-22 00:00:00
2022-11-08 00:00:00
2023-04-25 00:00:00
AGIXUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
AGLDUSDT
2020-01-01 00:00:00
2022-11-08 00:00:00
2023-04-25 00:00:00
AKROUSDT
2020-01-01 00:00:00
2022-12-11 00:00:00
2023-04-25 00:00:00
A

2023-02-28 00:00:00
2023-04-25 00:00:00
HOOKUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
HOTUSDT
2020-01-01 00:00:00
2022-10-14 00:00:00
2023-04-25 00:00:00
ICPUSDT
2020-01-01 00:00:00
2022-04-03 00:00:00
2022-10-20 00:00:00
2023-04-25 00:00:00
ICXUSDT
2020-01-01 00:00:00
2022-10-30 00:00:00
2023-04-25 00:00:00
IDUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
ILVUSDT
2020-01-01 00:00:00
2022-06-14 00:00:00
2022-12-31 00:00:00
2023-04-25 00:00:00
IMXUSDT
2020-01-01 00:00:00
2022-06-12 00:00:00
2022-12-29 00:00:00
2023-04-25 00:00:00
INJUSDT
2020-01-01 00:00:00
2023-03-05 00:00:00
2023-04-25 00:00:00
IOSTUSDT
2020-01-01 00:00:00
2022-04-30 00:00:00
2022-11-16 00:00:00
2023-04-25 00:00:00
IOTAUSDT
2020-01-01 00:00:00
2022-07-17 00:00:00
2023-02-02 00:00:00
2023-04-25 00:00:00
IOTXUSDT
2020-01-01 00:00:00
2022-05-31 00:00:00
2022-12-17 00:00:00
2023-04-25 00:00:00
JASMYUSDT
2020-01-01 00:00:00
2022-08-07 00:00:00
2023-02-23 00:00:00
2023-04-25 00:00:00
JOEUSDT
2020-01-01 00:00:00
2023-04-25 

2022-12-30 00:00:00
2023-04-25 00:00:00
YFIUSDT
2020-01-01 00:00:00
2022-06-28 00:00:00
2023-01-14 00:00:00
2023-04-25 00:00:00
YGGUSDT
2020-01-01 00:00:00
2022-06-11 00:00:00
2022-12-28 00:00:00
2023-04-25 00:00:00
ZECUSDT
2020-01-01 00:00:00
2022-06-12 00:00:00
2022-12-29 00:00:00
2023-04-25 00:00:00
ZENUSDT
2020-01-01 00:00:00
2022-06-14 00:00:00
2022-12-31 00:00:00
2023-04-25 00:00:00
ZILUSDT
2020-01-01 00:00:00
2022-08-18 00:00:00
2023-03-06 00:00:00
2023-04-25 00:00:00
ZRXUSDT
2020-01-01 00:00:00
2022-10-14 00:00:00
2023-04-25 00:00:00


In [23]:
# hourly data
download_crypto_data_v5(basicinfo=basicinfo,
                        assettype='futures',
                        saving_folder='C:\\Users\\paperspace\\Documents\\wmlakefund\\data\\v5\\futures\\hourly\\', 
                        sdate='2020-01-01', 
                        interval='D', 
                        sindex=0,
                        tindex=len(basicinfo),
                        returndict=False)

10000NFTUSDT
2020-01-01 00:00:00
2022-08-12 00:00:00
2023-02-28 00:00:00
2023-04-25 00:00:00
1000BONKUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
1000BTTUSDT
2020-01-01 00:00:00
2022-09-13 00:00:00
2023-04-01 00:00:00
2023-04-25 00:00:00
1000FLOKIUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
1000LUNCUSDT
2020-01-01 00:00:00
2023-03-28 00:00:00
2023-04-25 00:00:00
1000XECUSDT
2020-01-01 00:00:00
2022-10-30 00:00:00
2023-04-25 00:00:00
1INCHUSDT
2020-01-01 00:00:00
2022-06-24 00:00:00
2023-01-10 00:00:00
2023-04-25 00:00:00
AAVEUSDT
2020-01-01 00:00:00
2021-11-29 00:00:00
2022-06-17 00:00:00
2023-01-03 00:00:00
2023-04-25 00:00:00
ACHUSDT
2020-01-01 00:00:00
2022-10-25 00:00:00
2023-04-25 00:00:00
ADAUSDT
2020-01-01 00:00:00
2021-10-04 00:00:00
2022-04-22 00:00:00
2022-11-08 00:00:00
2023-04-25 00:00:00
AGIXUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
AGLDUSDT
2020-01-01 00:00:00
2022-11-08 00:00:00
2023-04-25 00:00:00
AKROUSDT
2020-01-01 00:00:00
2022-12-11 00:00:00
2023-04-25 00:00:00
A

2023-02-28 00:00:00
2023-04-25 00:00:00
HOOKUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
HOTUSDT
2020-01-01 00:00:00
2022-10-14 00:00:00
2023-04-25 00:00:00
ICPUSDT
2020-01-01 00:00:00
2022-04-03 00:00:00
2022-10-20 00:00:00
2023-04-25 00:00:00
ICXUSDT
2020-01-01 00:00:00
2022-10-30 00:00:00
2023-04-25 00:00:00
IDUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
ILVUSDT
2020-01-01 00:00:00
2022-06-14 00:00:00
2022-12-31 00:00:00
2023-04-25 00:00:00
IMXUSDT
2020-01-01 00:00:00
2022-06-12 00:00:00
2022-12-29 00:00:00
2023-04-25 00:00:00
INJUSDT
2020-01-01 00:00:00
2023-03-05 00:00:00
2023-04-25 00:00:00
IOSTUSDT
2020-01-01 00:00:00
2022-04-30 00:00:00
2022-11-16 00:00:00
2023-04-25 00:00:00
IOTAUSDT
2020-01-01 00:00:00
2022-07-17 00:00:00
2023-02-02 00:00:00
2023-04-25 00:00:00
IOTXUSDT
2020-01-01 00:00:00
2022-05-31 00:00:00
2022-12-17 00:00:00
2023-04-25 00:00:00
JASMYUSDT
2020-01-01 00:00:00
2022-08-07 00:00:00
2023-02-23 00:00:00
2023-04-25 00:00:00
JOEUSDT
2020-01-01 00:00:00
2023-04-25 

2022-12-30 00:00:00
2023-04-25 00:00:00
YFIUSDT
2020-01-01 00:00:00
2022-06-28 00:00:00
2023-01-14 00:00:00
2023-04-25 00:00:00
YGGUSDT
2020-01-01 00:00:00
2022-06-11 00:00:00
2022-12-28 00:00:00
2023-04-25 00:00:00
ZECUSDT
2020-01-01 00:00:00
2022-06-12 00:00:00
2022-12-29 00:00:00
2023-04-25 00:00:00
ZENUSDT
2020-01-01 00:00:00
2022-06-14 00:00:00
2022-12-31 00:00:00
2023-04-25 00:00:00
ZILUSDT
2020-01-01 00:00:00
2022-08-18 00:00:00
2023-03-06 00:00:00
2023-04-25 00:00:00
ZRXUSDT
2020-01-01 00:00:00
2022-10-14 00:00:00
2023-04-25 00:00:00


In [24]:
# minute bar data
download_crypto_data_v5(basicinfo=basicinfo,
                        assettype='futures',
                        saving_folder='C:\\Users\\paperspace\\Documents\\wmlakefund\\data\\v5\\futures\\minute_bar\\', 
                        sdate='2020-01-01', 
                        interval='D', 
                        sindex=0,
                        tindex=len(basicinfo),
                        returndict=False)

10000NFTUSDT
2020-01-01 00:00:00
2022-08-12 00:00:00
2023-02-28 00:00:00
2023-04-25 00:00:00
1000BONKUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
1000BTTUSDT
2020-01-01 00:00:00
2022-09-13 00:00:00
2023-04-01 00:00:00
2023-04-25 00:00:00
1000FLOKIUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
1000LUNCUSDT
2020-01-01 00:00:00
2023-03-28 00:00:00
2023-04-25 00:00:00
1000XECUSDT
2020-01-01 00:00:00
2022-10-30 00:00:00
2023-04-25 00:00:00
1INCHUSDT
2020-01-01 00:00:00
2022-06-24 00:00:00
2023-01-10 00:00:00
2023-04-25 00:00:00
AAVEUSDT
2020-01-01 00:00:00
2021-11-29 00:00:00
2022-06-17 00:00:00
2023-01-03 00:00:00
2023-04-25 00:00:00
ACHUSDT
2020-01-01 00:00:00
2022-10-25 00:00:00
2023-04-25 00:00:00
ADAUSDT
2020-01-01 00:00:00
2021-10-04 00:00:00
2022-04-22 00:00:00
2022-11-08 00:00:00
2023-04-25 00:00:00
AGIXUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
AGLDUSDT
2020-01-01 00:00:00
2022-11-08 00:00:00
2023-04-25 00:00:00
AKROUSDT
2020-01-01 00:00:00
2022-12-11 00:00:00
2023-04-25 00:00:00
A

2023-02-28 00:00:00
2023-04-25 00:00:00
HOOKUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
HOTUSDT
2020-01-01 00:00:00
2022-10-14 00:00:00
2023-04-25 00:00:00
ICPUSDT
2020-01-01 00:00:00
2022-04-03 00:00:00
2022-10-20 00:00:00
2023-04-25 00:00:00
ICXUSDT
2020-01-01 00:00:00
2022-10-30 00:00:00
2023-04-25 00:00:00
IDUSDT
2020-01-01 00:00:00
2023-04-25 00:00:00
ILVUSDT
2020-01-01 00:00:00
2022-06-14 00:00:00
2022-12-31 00:00:00
2023-04-25 00:00:00
IMXUSDT
2020-01-01 00:00:00
2022-06-12 00:00:00
2022-12-29 00:00:00
2023-04-25 00:00:00
INJUSDT
2020-01-01 00:00:00
2023-03-05 00:00:00
2023-04-25 00:00:00
IOSTUSDT
2020-01-01 00:00:00
2022-04-30 00:00:00
2022-11-16 00:00:00
2023-04-25 00:00:00
IOTAUSDT
2020-01-01 00:00:00
2022-07-17 00:00:00
2023-02-02 00:00:00
2023-04-25 00:00:00
IOTXUSDT
2020-01-01 00:00:00
2022-05-31 00:00:00
2022-12-17 00:00:00
2023-04-25 00:00:00
JASMYUSDT
2020-01-01 00:00:00
2022-08-07 00:00:00
2023-02-23 00:00:00
2023-04-25 00:00:00
JOEUSDT
2020-01-01 00:00:00
2023-04-25 

2022-12-30 00:00:00
2023-04-25 00:00:00
YFIUSDT
2020-01-01 00:00:00
2022-06-28 00:00:00
2023-01-14 00:00:00
2023-04-25 00:00:00
YGGUSDT
2020-01-01 00:00:00
2022-06-11 00:00:00
2022-12-28 00:00:00
2023-04-25 00:00:00
ZECUSDT
2020-01-01 00:00:00
2022-06-12 00:00:00
2022-12-29 00:00:00
2023-04-25 00:00:00
ZENUSDT
2020-01-01 00:00:00
2022-06-14 00:00:00
2022-12-31 00:00:00
2023-04-25 00:00:00
ZILUSDT
2020-01-01 00:00:00
2022-08-18 00:00:00
2023-03-06 00:00:00
2023-04-25 00:00:00
ZRXUSDT
2020-01-01 00:00:00
2022-10-14 00:00:00
2023-04-25 00:00:00


In [1]:
import sys

In [2]:
sys.executable

'C:\\Users\\charles_huang\\anaconda3\\python.exe'